# Shakedown IPython Demo

## Load, override and show config

In [1]:
%%sdconfig --show ../config.yml
vars:
  interfaces: ["Et1 - 2"]

connections:
  veos-2:
    creds: [admin, '']
    protocol: eapi+http
    tags: [dut, tor]
  veos-3:
    creds: [admin, '']
    tags: [sdut, csp]
settings: {auto_rollback: true, raise_on_error: true}
tests:
  basics: {software_version: 4.20.5F}
  downgrade: {testrail_case_id: C12345}
  ntp:
    servers: [192.168.56.10]
vars:
  interfaces: [Et1 - 2]
  vrf: management


## Send commands to DUTs

### Regex hostname filter

In [2]:
%%sdsend veos-.
show hostname

host: veos-3
status: ok
commands:
  - command: {'cmd': 'show hostname', 'input': ''}
    status: ok
    output: |
      Hostname: veos-3
      FQDN:     veos-3
host: veos-2
status: ok
commands:
  - command: {'cmd': 'show hostname', 'input': ''}
    status: ok
    output: |
      Hostname: veos-2
      FQDN:     veos-2


### Filter by tags

In [3]:
%%sdsend csp tor
show ip interface brief

host: veos-2
status: ok
commands:
  - command: {'cmd': 'show ip interface brief', 'input': ''}
    status: ok
    output: |
      Interface              IP Address         Status     Protocol         MTU
      Ethernet2              10.0.23.0/31       up         up              1500
      Loopback0              10.0.0.2/32        up         up             65535
      Management1            192.168.56.12/24   up         up              1500
host: veos-3
status: ok
commands:
  - command: {'cmd': 'show ip interface brief', 'input': ''}
    status: ok
    output: |
      Interface              IP Address         Status     Protocol         MTU
      Ethernet1              10.0.23.1/31       up         up              1500
      Ethernet2              10.0.34.0/31       up         up              1500
      Ethernet3              unassigned         admin down down            1500
      Loopback0              10.0.0.3/32        up         up             65535
      Management1            192

## Access session from Python

In [ ]:
for resp in _sdsessions.send(".*", "show version"):
    print(resp)

## Jinja templates are supported

In [ ]:
%%sdsend .*
{% for intf in vars.interfaces %}
show interfaces {{intf}} status
{% endfor %}

## Run bash commands on switch...

In [ ]:
%%sdsend dut
bash timeout 5 ps -ef | grep Sand

## Write output to file

In [ ]:
%%sdsend dut -o clock.txt
show clock